# Install Dependencies

In [17]:
!pip3 install gradientai --upgrade

Requirement already up-to-date: gradientai in /home/razzuren/.local/lib/python3.8/site-packages (1.11.0)


# Set env properties, instantiate model

In [1]:
import os
import constants
os.environ['GRADIENT_ACCESS_TOKEN'] = constants.GRADIENT_ACCESS_TOKEN
os.environ['GRADIENT_WORKSPACE_ID'] = constants.GRADIENT_WORKSPACE_ID
from gradientai import Gradient

with Gradient() as gradient:
  base_model = gradient.get_base_model(base_model_slug="nous-hermes2")
  model_motherofgod = base_model.create_model_adapter(
      name="model-motherofgod-v1"
  )
  rag_collection = gradient.create_rag_collection(
  name="Simple with text files",
  slug="bge-large",
  filepaths=[
    "data/data.txt",
  ],
)
  print("Set")

Set


# Pure Retrieval

In [13]:
print(f"Created model adapter with id {model_motherofgod.id}")
simpler_query = """
Write a descriptive anamnesis in Brazilian Portuguese for this pacient based on following data
- Sexo: Feminino
- Idade: 68 anos
- Etnia: Branca
- Queixa Atual:
  - Tosse produtiva com escarro hialino
  - Dispneia aos médios esforços há mais de 20 anos
  - Piora da dispneia há 2 meses, manifestando-se ao repouso e em posição ortostática
  - Nega febre e alteração da tosse
- Histórico Pessoal:
  - Tabagista de um maço por dia há 60 anos
  - Dois episódios de pneumonia, sendo o último há 2 semanas (com tratamento domiciliar)
  - Nega hipertensão arterial sistêmica
  - Nega uso de medicamentos
- Exame Físico:
  - Cor:
    - Cianose: ++/+++
    - Descorada: +
  - Pressão Arterial: 90x50 mmHg
  - Pulso: 100 bpm
  - Frequência Respiratória: 37 movimentos/min
  - Temperatura: 36,5°C (afebril)
  - Estase jugular: Presente a 45°
  - Oictus cordis: Não palpável
  - Ausculta cardíaca:
    - Sopro sistólico + em foco mitral, aumentado à manobra de Valsalva
    - Sopro sistólico ++ em foco tricúspide, aumentado à manobra de Rivero-Carvalho
  - Tórax:
    - Aumento do diâmetro ântero-posterior
    - Percussão: Som hipersonoro
    - Murmúrio vesicular: Diminuído globalmente
    - Estertores subcrepitantes: Presentes em bases e campos médios
  - Abdome:
    - Fígado: Palpável a 3 cm do arco costal em linha hemiclavicular direita, endurecido + e indolor
  - Edema de membros inferiores: Ausente
- Exames laboratoriais:
  - Hemograma:
    - Eritrócitos: 3,56 x 10^6/mm³
    - Hemoglobina: 11,2 g/dL
    - Hematócrito: 33,4%
    - VCM: 94 µm³
    - Plaquetas: 283.000 /mm³
    - Leucócitos: 8.800 /mm³
      - Metamielócitos: 88
      - Bastonetes: 528
      - Segmentados: 5.632
      - Eosinófilos: 264
      - Linfócitos típicos: 1.408
      - Monócitos: 880
  - Coagulograma:
    - Tempo de protrombina: 14,9 s
    - Atividade de protrombina: 100%
    - RNI: 1,00
    - TTPA: 37,2 s
    - Testemunho: 30,8 s
    - R: 1,2
  - Urina tipo I:
    - Nitrito: Positivo
    - Hemácias: 500 /mL (normal até 3.500/mL)
    - Leucócitos: 168.000 /mL (normal até 6.500/mL)
    - Bactérias: Numerosas
    - Cultura: Positiva para E. coli, sensível a ciprofloxacina
- Encaminhamento: Solicitar anamnese de ECO.
"""
#print(f"Asking: {simpler_query}")

completion = gradient.answer(question=simpler_query,
                             source={
                                "collectionId" : rag_collection.id_,
                                "modelId": model_motherofgod.id,
                                 "type":"rag"
                             })['answer']

print(f"Generated: \n {completion}")

Created model adapter with id b02e4449-a7a0-4219-884c-3e9594705b10_model_adapter
Generated: 
 Anamnese:

Paciente feminina de 68 anos, de etnia branca, relata tosse produtiva com escarro hialino, dispneia aos médios esforços há mais de 20 anos, com piora nos últimos 2 meses, manifestando-se ao repouso e em posição ortostática. Nega febre e alteração da tosse. Histórico pessoal de tabagista de um maço por dia há 60 anos, com dois episódios de pneumonia, sendo o último há 2 semanas, tratado em domicílio. Nega hipertensão arterial sistêmica e uso de medicamentos.

No exame físico, apresenta cor cianótica, pressão arterial 90x50 mmHg, pulso 100 bpm, frequência respiratória 37 movimentos/min, temperatura afebril, estase jugular presente a 45°, oictus cordis não palpável. Ausculta cardíaca com sopro sistólico + em foco mitral, aumentado à manobra de Valsalva, e sopro sistólico ++ em foco tricúspide, aumentado à manobra de Rivero-Carvalho. Tórax com aumento do diâmetro ântero-posterior, som h

# Fine Tunning using static

In [4]:
samples = []

# Define the directories where the files are located
in_directory = 'finetunning/prompts'
out_directory = 'finetunning/response'

# Ensure that there are equal numbers of 'in' and 'out' files
assert len(os.listdir(in_directory)) == len(os.listdir(out_directory)), "Mismatch in number of 'in' and 'out' files"

# Loop through all files in the 'in' directory
for filename in sorted(os.listdir(in_directory)):
    # Only process .txt files
    if filename.endswith('.txt'):
        # Determine the full path of the file
        in_filepath = os.path.join(in_directory, filename)

        # Open the file and read its content
        with open(in_filepath, 'r') as f:
            instruction = f.read().strip()

        # Determine the full path of the corresponding 'out' file
        out_filepath = os.path.join(out_directory, filename)

        # Open the 'out' file and read its content
        with open(out_filepath, 'r') as f:
            response = f.read().strip()

        # Add the sample to the list
        samples.append({
            "inputs": "### Instruction:" + instruction + " \n\n### Response: "  +response
        })

# this is where fine-tuning happens
num_epochs = 4
count = 0
while count < num_epochs:
  print(f"Fine-tuning the model, iteration {count + 1}")
  model_motherofgod.fine_tune(samples=samples)
  count = count + 1

Fine-tuning the model, iteration 1
Fine-tuning the model, iteration 2
Fine-tuning the model, iteration 3
Fine-tuning the model, iteration 4


# Format as MD

In [22]:
formatmd_query = f"""
### Instruction:
Format the following text as a Markdown file:
{completion}

### Response:
"""
print(f"Asking: {formatmd_query}")

formated_md = model_motherofgod.complete(query=sample_query, max_generated_token_count=500).generated_output
print(f"Generated: \n {formated_md}")

Asking: 
### Instruction:
Format the following text as a Markdown file:
 < NOME >, 53 anos, do sexo masculino, apresenta-se à consulta do pneumologista com queixa de dispnéia e dor torácica. Relata que há 4 dias apresentou fadiga, febre, calafrios e tosse produtiva, às vezes com eliminação de escarro esverdeado. Dois dias após o início do quadro, a dispnéia tornou-se mais grave e a febre elevada (TA: 39,5º C). No momento, queixa-se de dor torácica, em pontada, na região inferior do hemitórax esquerdo, que se acentua com a tosse e à inspiração. Nega tabagismo. Relata ser diabético há dois anos.

Paciente em exame físico apresenta-se com PA: 140 x 90 mm Hg, FR: 32 irpm e TA: 40ºC. É avaliado como em bom estado geral, corado, hidratado, anictérico, sianoótico, sem linfadenomegalias. AR: crepitações na região inferior do hemitórax esquerdo. ACV: bulhas taquicárdias e normofonéticas. AD e AGU: ndn.

Solicitados exames laboratoriais: BAAR + RX TORAX + LABS ( TALVEZ TESTE CUTANEO)

Encaminham

# Retrieve from RAG

In [8]:
print(f"Mmodel adapter with RAG {rag_collection}")
simpler_query = """
Write a descriptive anamnesis in Brazilian Portuguese for this pacient based on following data
- Sexo: Feminino
- Idade: 68 anos
- Etnia: Branca
- Queixa Atual:
  - Tosse produtiva com escarro hialino
  - Dispneia aos médios esforços há mais de 20 anos
  - Piora da dispneia há 2 meses, manifestando-se ao repouso e em posição ortostática
  - Nega febre e alteração da tosse
- Histórico Pessoal:
  - Tabagista de um maço por dia há 60 anos
  - Dois episódios de pneumonia, sendo o último há 2 semanas (com tratamento domiciliar)
  - Nega hipertensão arterial sistêmica
  - Nega uso de medicamentos
- Exame Físico:
  - Cor:
    - Cianose: ++/+++
    - Descorada: +
  - Pressão Arterial: 90x50 mmHg
  - Pulso: 100 bpm
  - Frequência Respiratória: 37 movimentos/min
  - Temperatura: 36,5°C (afebril)
  - Estase jugular: Presente a 45°
  - Oictus cordis: Não palpável
  - Ausculta cardíaca:
    - Sopro sistólico + em foco mitral, aumentado à manobra de Valsalva
    - Sopro sistólico ++ em foco tricúspide, aumentado à manobra de Rivero-Carvalho
  - Tórax:
    - Aumento do diâmetro ântero-posterior
    - Percussão: Som hipersonoro
    - Murmúrio vesicular: Diminuído globalmente
    - Estertores subcrepitantes: Presentes em bases e campos médios
  - Abdome:
    - Fígado: Palpável a 3 cm do arco costal em linha hemiclavicular direita, endurecido + e indolor
  - Edema de membros inferiores: Ausente
- Exames laboratoriais:
  - Hemograma:
    - Eritrócitos: 3,56 x 10^6/mm³
    - Hemoglobina: 11,2 g/dL
    - Hematócrito: 33,4%
    - VCM: 94 µm³
    - Plaquetas: 283.000 /mm³
    - Leucócitos: 8.800 /mm³
      - Metamielócitos: 88
      - Bastonetes: 528
      - Segmentados: 5.632
      - Eosinófilos: 264
      - Linfócitos típicos: 1.408
      - Monócitos: 880
  - Coagulograma:
    - Tempo de protrombina: 14,9 s
    - Atividade de protrombina: 100%
    - RNI: 1,00
    - TTPA: 37,2 s
    - Testemunho: 30,8 s
    - R: 1,2
  - Urina tipo I:
    - Nitrito: Positivo
    - Hemácias: 500 /mL (normal até 3.500/mL)
    - Leucócitos: 168.000 /mL (normal até 6.500/mL)
    - Bactérias: Numerosas
    - Cultura: Positiva para E. coli, sensível a ciprofloxacina
- Encaminhamento: Solicitar anamnese de ECO.
"""
#print(f"Asking: {simpler_query}")

completion = gradient.answer(question=simpler_query,
                             source={
                                "type": "rag",
                                "collectionId" : rag_collection.id_,
                                "modelId": model_motherofgod.id
                             })['answer']

print(f"Generated: \n {completion}")

Mmodel adapter with RAG <gradientai._rag.RAGCollection object at 0x7faf3d364410>
Generated: 
 Anamnese:

Paciente feminina de 68 anos, de etnia branca, relata tosse produtiva com escarro hialino, dispneia aos médios esforços há mais de 20 anos, com piora nos últimos 2 meses, manifestando-se ao repouso e em posição ortostática. Nega febre e alteração da tosse. Histórico pessoal de tabagista de um maço por dia há 60 anos, com dois episódios de pneumonia, sendo o último há 2 semanas, tratado em domicílio. Nega hipertensão arterial sistêmica e uso de medicamentos.

No exame físico, apresenta cor cianótica, pressão arterial 90x50 mmHg, pulso 100 bpm, frequência respiratória 37 movimentos/min, temperatura afebril, estase jugular presente a 45°, oictus cordis não palpável. Ausculta cardíaca com sopro sistólico + em foco mitral, aumentado à manobra de Valsalva, e sopro sistólico ++ em foco tricúspide, aumentado à manobra de Rivero-Carvalho. Tórax com aumento do diâmetro ântero-posterior, som h

In [34]:
print(f"Created model adapter with id {model_motherofgod.id}")
sample_query = """
### Instruction:
What will I do in 27th apr?

### Response:
"""
print(f"Asking: {sample_query}")

completion = model_motherofgod.complete(query=sample_query, max_generated_token_count=500).generated_output
print(f"Generated: \n {completion}")

Created model adapter with id fb877bb2-39bc-488a-8b82-007aec5c746f_model_adapter
Asking: 
### Instruction:
What will I do in 27th april?

### Response:

Generated: 
  I am sorry, but I do not have a crystal ball to see the future. Can you please provide me with more information about the 27th of April? Is there a specific event or task that you want me to help you plan for?
